# Facial Expression Recognition

In this notebook, we will explore the facial expression recognition challenge.


In [ ]:
from classifier import train_classifier, eval_classifier

import torch
import torch.nn as nn
import torchvision
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import copy
import kagglehub

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Dataset


In [ ]:
# Download FER-2013 latest version
data_dir = kagglehub.dataset_download("msambare/fer2013")

data_transforms = torchvision.transforms.Compose(
    [
        torchvision.transforms.Grayscale(
            num_output_channels=3
        ),  # si modèles pré-entraînés ImageNet
        torchvision.transforms.Resize((224, 224)),
        torchvision.transforms.ToTensor(),
    ]
)

# Load the FER-2013 dataset
train_data = torchvision.datasets.ImageFolder(
    data_dir + "/train", transform=data_transforms
)

test_data = torchvision.datasets.ImageFolder(
    data_dir + "/test", transform=data_transforms
)

print("Classes of the dataset:", train_data.classes)
print("Number of training samples:", len(train_data))
print("Number of test samples:", len(test_data))

# Pour afficher le mapping complet
for idx, emotion in enumerate(train_data.classes):
    print(f"Label {idx} → {emotion}")

In [ ]:
batch_size = 16

train_dataloader = DataLoader(
    train_data, batch_size=batch_size, shuffle=True, drop_last=True
)

test_dataloader = DataLoader(
    test_data, batch_size=batch_size, shuffle=True, drop_last=True
)

# - print the number of batches in the training subset
num_batches = len(train_dataloader)
print("Number of batches in the training subset:", num_batches)

# - print the number of batches in the testing subset
num_batches = len(test_dataloader)
print("Number of batches in the testing subset:", num_batches)

In [ ]:
# 3. Calcul de mean & std
sum_ = 0.0
sum_sq_ = 0.0
nb_pixels = 0

for imgs, _ in test_dataloader:
    # imgs shape: (B,1,48,48)
    B, C, H, W = imgs.shape
    imgs = imgs.view(B, C, -1)  # (B,1,2304)
    sum_ += imgs.sum(dim=[0, 2])  # somme des pixels par canal
    sum_sq_ += (imgs**2).sum(dim=[0, 2])  # somme des carrés
    nb_pixels += B * H * W

mean = sum_ / nb_pixels  # tensor([μ])
std = torch.sqrt(
    sum_sq_ / torch.tensor(nb_pixels, dtype=torch.float32) - mean**2
)  # tensor([σ])

print("Dataset mean:", mean)  # ex. ≈tensor([0.485])
print("Dataset std: ", std)  # ex. ≈tensor([0.237])

In [ ]:
data_transforms = torchvision.transforms.Compose(
    [
        torchvision.transforms.Grayscale(
            num_output_channels=3
        ),  # si modèles pré-entraînés ImageNet
        torchvision.transforms.Resize((224, 224)),  # ou autre résolution
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(mean=mean, std=std),
    ]
)

# Load the FER-2013 dataset
train_data = torchvision.datasets.ImageFolder(
    data_dir + "/train", transform=data_transforms
)

test_data = torchvision.datasets.ImageFolder(
    data_dir + "/test", transform=data_transforms
)

print("Classes of the dataset:", train_data.classes)
print("Number of training samples:", len(train_data))
print("Number of test samples:", len(test_data))

# Pour afficher le mapping complet
for idx, emotion in enumerate(train_data.classes):
    print(f"Label {idx} → {emotion}")

In [ ]:
batch_size = 16

train_dataloader = DataLoader(
    train_data, batch_size=batch_size, shuffle=True, drop_last=True
)

test_dataloader = DataLoader(
    test_data, batch_size=batch_size, shuffle=True, drop_last=True
)

# - print the number of batches in the training subset
num_batches = len(train_dataloader)
print("Number of batches in the training subset:", num_batches)

# - print the number of batches in the testing subset
num_batches = len(test_dataloader)
print("Number of batches in the testing subset:", num_batches)

In [ ]:
weights = torchvision.models.VGG11_Weights.IMAGENET1K_V1
model = torchvision.models.vgg11(
    weights=weights
)  # charges les poids ImageNet pré-entraînés

# Geler toutes les couches
for param in model.parameters():
    param.requires_grad = (
        False  # figure l’intégralité des poids pour ne pas les recalculer
    )

# Remplacer la couche de sortie (le dernier module du classifier)
num_classes = 7
model.classifier[6] = nn.Linear(in_features=4096, out_features=num_classes)

# Cette nouvelle couche est dégelée par défaut, car elle vient d'être créée
# Mais pour être explicite:
for param in model.classifier[6].parameters():
    param.requires_grad = True

In [ ]:
num_epochs = 30
learning_rate = 0.01
loss_fn = nn.CrossEntropyLoss()

model_trained, train_losses = train_classifier(
    model=model,
    train_dataloader=train_dataloader,
    batch_size=batch_size,
    num_epochs=num_epochs,
    loss_fn=loss_fn,
    learning_rate=learning_rate,
    device=device,
    transform_fn=None,
    verbose=True,
)

torch.save(model_trained.state_dict(), "training/vgg-11_trained.pt")

## Testing the model on the test dataset


In [ ]:
model_test = copy.deepcopy(model)
model_test.load_state_dict(torch.load("training/vgg-11_trained.pt"))

# - Apply the evaluation function using the test dataloader
test_accuracy = eval_classifier(
    model=model_test, eval_dataloader=test_dataloader, device=device
)

# - Print the test accuracy
print("Test accuracy: {:.2f}%".format(test_accuracy))

# - Plot the training loss over epochs
plt.figure()
plt.plot(train_losses)
plt.title("Training loss over epochs")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.grid()
plt.show()